# Arrays

## Features

- Limit capacity (defined at creation time)
- Can not modify it's shape
- Can not add or remove positions
- Could have multiple dimensions (1, 2, 3... n)
- Arrays are a type of lists, but lists are not a type of arrays
- Python has an Array module but is based on lists, so it does not allow storing all types of data

In [1]:
import random
from functools import reduce

### 1D Array

In [2]:
class Array:
    
    def __init__(self, capacity, fill_value=None):
        self.items = list()

        # Set default items
        for i in range(capacity):
            self.items.append(fill_value)

    def __len__(self):
        return len(self.items)

    def __str__(self):
        return str(self.items)

    def __iter__(self):
        return iter(self.items)

    def __getitem__(self, index):
        return self.items[index]

    def __setitem__(self, index, new_item):
        self.items[index] = new_item

    def __populate__(self, min_value, max_value):
        for i in range(self.__len__()):
             self.__setitem__(i, random.randint(min_value, max_value))
        return self.__str__()

    def __sum__(self):
        return reduce(lambda current_item, next_item: current_item+next_item, self.items)

In [3]:
array = Array(5)

In [4]:
array.__populate__(5,100)

'[19, 58, 75, 38, 77]'

In [5]:
len(array)

5

In [6]:
str(array)

'[19, 58, 75, 38, 77]'

In [27]:
array[0]

0

In [7]:
for i in range(5):
    array[i] = i
print(array)

[0, 1, 2, 3, 4]


In [8]:
array[4]

4

In [9]:
[i for i in array.__iter__()]

[0, 1, 2, 3, 4]

### 2D Array

In [10]:
class Grid():

    def __init__(self, rows, columns, fill_value=None):
        # Defines the rows of the array
        self.data = Array(rows)

        # Defines the columns of the array and set their default values
        for row in range(rows):
            self.data[row] = Array(columns, fill_value=fill_value)

    def get_height(self):
        return len(self.data)

    def get_width(self):
        return len(self.data[0])

    def __setitem__(self, row, column, value):
        self.data[row][column] = value

    def __getitem__(self, row, column):
        return self.data[row][column]

    def __str__(self):
        string = ""

        for row in range(self.get_height()):
            for col in range(self.get_width()):
                string += str(self.data[row][col]) + " "

            string += "\n"

        return str(string)

In [32]:
grid = Grid(3, 3)

In [33]:
grid.__populate__(1, 100)

57 58 28 
36 93 42 
72 42 61 



In [34]:
grid.__setitem__(1, 1, '(1,1)')
print(grid)

57 58 28 
36 (1,1) 42 
72 42 61 



In [38]:
value = grid.__getitem__(0, 1)
value

58

# Drawing tool

In [43]:
class Canvas:

    def __init__(self, width, height, fill_value='|_|', mark='x'):
        self.background = fill_value
        self.mark = mark

        self.height = height
        self.width = width

        self.borders = 2
        self.canvas = Grid(height+self.borders, width+self.borders, fill_value)
        self.__draw_borders()

    def __str__(self):
        return str(self.canvas)

    def __setitem__(self, row, col, value):
        self.canvas.__setitem__(row, col, value)

    def __getitem__(self, row, col):
        return self.canvas.__getitem__(row, col)

    def __draw_borders(self):
        ''' Draws the borders of the Canvas with horizontal and vertical lines '''
        last_row = self.height + 1
        last_col = self.width + 1

        # Draw horizontal lines
        horizontal_edge = '-'
        self.__draw_horizontal_line(0, 0, 0, last_col, horizontal_edge)
        self.__draw_horizontal_line(last_row, 0, last_row, last_col, horizontal_edge)

        # Draw vertical lines
        vertical_edge = '|'
        self.__draw_vertical_line(1, 0, self.height, 0, vertical_edge)
        self.__draw_vertical_line(1, last_col, self.height, last_col, vertical_edge)

    def __draw_horizontal_line(self, row1, col1, row2, col2, value):
        ''' Iterate over the columns setting the specified value '''
        for col in range(col1, col2+1):
            self.canvas.__setitem__(row1, col, value)

    def __draw_vertical_line(self, row1, col1, row2, col2, value):
        ''' Iterate over the rows setting the specified value '''
        for row in range(row1, row2+1):
            self.canvas.__setitem__(row, col1, value)

    def create_line(self, row1, col1, row2, col2, value=None):
        '''
        Should create a new line from (row1,col1) to (row2,col2).
        Currently only horizontal or vertical lines are supported.
        '''
        value = value or self.mark

        # Horizontal condition
        if row1==row2:

            # Validates vertical limits
            if self.height >= row1 > 0:

                # Sorting the columns to allow drawing in two directions
                if col1 > col2:
                    col2, col1 = col1, col2

                # If the col1 is less than or equal to 0, is setted to the minimum value
                if col1 <= 0:
                    col1 = 1

                # Set col2 to the canvas width if it exceeds horizontal limits
                if col2 > self.width:
                    col2 = self.width

                self.__draw_horizontal_line(row1, col1, row2, col2, value)

        # Vertical condition
        elif col1==col2:

            # Validates horizontal limits
            if self.width >= col1 > 0:

                # Sorting the rows to allow drawing in two directions
                if row1 > row2:
                    row2, row1 = row1, row2

                # If the row1 is less than or equal to 0, is setted to the minimum value
                if row1 <= 0:
                    row1 = 1

                # Set row2 to the canvas width if it exceeds vertical limits
                if row2 > self.height:
                    row2 = self.height

                self.__draw_vertical_line(row1, col1, row2, col2, value)

        else:
            raise OnlyStraightLinesAllowed

    def create_rectangle(self, row1, col1, row2, col2, value=None):
        '''
        Should create a new rectangle, whose upper left corner is (row1,col1)
        and lower right corner is (row2,col2).
        '''
        value = value or self.mark

        # Draw horizontal lines
        self.create_line(row1, col1, row1, col2, value)
        self.create_line(row2, col1, row2, col2, value)

        # Draw vertical lines
        self.create_line(row1, col1, row2, col1, value)
        self.create_line(row1, col2, row2, col2, value)

    def bucket_fill(self, row, col, color):
        '''
        Should fill the entire area connected to (row, col) with the specified "colour".
        The behaviour of this is the same as that of the "bucket fill" tool in paint programs.
        '''

        # Return if current position is already filled
        current = self.canvas.__getitem__(row, col)
        if current == self.mark:
            return
        # Set the current position to the new value if it is free
        elif current == self.background:
            self.canvas.__setitem__(row, col, color)
        else:
            return

        if row + 1 <= self.height:
            self.bucket_fill(row+1, col, color)
        if row - 1 > 0:
            self.bucket_fill(row-1, col, color)

        if col + 1 <= self.width:
            self.bucket_fill(row, col+1, color)
        if col - 1 > 0:
            self.bucket_fill(row, col-1, color)

## Create the canvas

In [44]:
w, h = 3, 3
canvas = Canvas(w, h, '|_|')
print(canvas)

- - - - - 
| |_| |_| |_| | 
| |_| |_| |_| | 
| |_| |_| |_| | 
- - - - - 



In [47]:
canvas.__getitem__(1, 0)

'|'

## Draw lines

In [16]:
value = '|X|'

In [17]:
x1, y1 = 1, 2
x2, y2 = 6, 2
row1, col1 = y1, x1
row2, col2 = y2, x2

canvas.create_line(row1, col1, row2, col2, value)
print(canvas)

| - - - - - - - - - - - - - - - - - - - - | 
| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| |X| |X| |X| |X| |X| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| - - - - - - - - - - - - - - - - - - - - | 



In [18]:
x1, y1 = 6, 3
x2, y2 = 6, 4
row1, col1 = y1, x1
row2, col2 = y2, x2

canvas.create_line(row1, col1, row2, col2, value)
print(canvas)

| - - - - - - - - - - - - - - - - - - - - | 
| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| |X| |X| |X| |X| |X| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| |_| |_| |_| |_| |_| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| |_| |_| |_| |_| |_| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| - - - - - - - - - - - - - - - - - - - - | 



## Draw a rectangle

In [19]:
x1, y1 = 16, 1
x2, y2 = 20, 3
row1, col1 = y1, x1
row2, col2 = y2, x2

canvas.create_rectangle(row1, col1, row2, col2, value)
print(canvas)

| - - - - - - - - - - - - - - - - - - - - | 
| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |X| |X| |X| |X| |X| | 
| |X| |X| |X| |X| |X| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |X| |_| |_| |_| |X| | 
| |_| |_| |_| |_| |_| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |X| |X| |X| |X| |X| | 
| |_| |_| |_| |_| |_| |X| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| |_| | 
| - - - - - - - - - - - - - - - - - - - - | 



## Bucket fill

In [20]:
x, y = 10, 3
row, col = y, x
color = '|O|'

canvas.bucket_fill(row, col, color)
print(canvas)

| - - - - - - - - - - - - - - - - - - - - | 
| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |X| |X| |X| |X| |X| | 
| |X| |X| |X| |X| |X| |X| |O| |O| |O| |O| |O| |O| |O| |O| |O| |X| |_| |_| |_| |X| | 
| |_| |_| |_| |_| |_| |X| |O| |O| |O| |O| |O| |O| |O| |O| |O| |X| |X| |X| |X| |X| | 
| |_| |_| |_| |_| |_| |X| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| |O| | 
| - - - - - - - - - - - - - - - - - - - - | 

